In [54]:
# This file consists of using built in native code executor of google ADK

In [55]:
# Importing basic libraries
from google.adk.agents import LlmAgent, SequentialAgent
from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
import os
from dotenv import load_dotenv

print("Libraries imported successfully")

Libraries imported successfully


In [56]:
# Checking configuration
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    print("API KEY FOUND")
else:
    print("CONFIGURATION ERROR")

API KEY FOUND


In [57]:
# Retry config function
def create_retry_config():
    '''
    This function returns the basic retry configuration for the AI agent
    '''

    return types.HttpRetryOptions(
        attempts=3,
        initial_delay=1,
        exp_base=2,
        http_status_codes=[429, 500, 501, 503, 504]
    )

retry_config = create_retry_config()

In [58]:
# This helper functions prints the python code generated by the agent and the its generated result from the code executor tool

def show_python_code_and_result(response):
    for i in range(len(response)):
        # Checking if the response contains a valid function call result from the code executor
        if(
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code.result != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", "")
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])

print("Defined helper function")

Defined helper function


In [59]:
code_generator=LlmAgent(
    name="CodeGeneratorAgent",
    model="gemini-2.5-flash-lite",
    instruction='''You are an expert python code generator.
    Your task is to generate a python code snippet based on the input from the user..
    - ONLY output the generated code, no introduction or explaination
    ''',
    description="Python code generator agent that generates code based on the user request.",
    output_key="generated_code"
)

In [60]:
code_executor=LlmAgent(
    name="CodeExecutorAgent",
    model="gemini-2.5-flash-lite",
    instruction='''You are a code executor.
    - Execute the following code snippet {generated_code} with a example argument to demonstrate the working of the code.
    - Use the 'BuiltInCodeExecutor' tool to execute the code
    - ONLY output the generated response/returned value, without any description of code's working.
    ''',
    description="Runs the code snippet passed by the generator_agent and outputs the returned value as response",
    code_executor=BuiltInCodeExecutor(),
    output_key="executed_code"
)



In [61]:
root_agent=SequentialAgent(
    name="CodeGeneratorPipeline",
    sub_agents=[code_generator, code_executor],
)

runner = InMemoryRunner(agent=root_agent)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/arpit/devlopement/agentic-ai-google-kaggle/myvenv/lib/python3.9/site-packages/google/adk/agents", which implies app name "agents".


In [62]:
response = await runner.run_debug(
    "Generate a code snippet to caluclate factorial of a number."
)


 ### Created new session: debug_session_id

User > Generate a code snippet to caluclate factorial of a number.
CodeGeneratorAgent > ```python
def factorial(n):
    """Calculates the factorial of a non-negative integer."""
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

# Example usage:
num = 5
print(f"The factorial of {num} is {factorial(num)}")
```


CodeExecutorAgent > The factorial of 5 is 120

